In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [7]:
html_content = requests.get("https://stats.espncricinfo.com/ci/engine/records/team/match_results_year.html?class=2;id=202;type=decade")
soup = BeautifulSoup(html_content.content,"html.parser")

In [13]:
ul_tag = soup.find("ul",class_ = "category-noindent")
a_tag = ul_tag.find_all("a")[2:]
year_links = ["https://stats.espncricinfo.com" + a['href'] for a in a_tag]
print(year_links)

['https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id=2022;type=year', 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id=2023;type=year']


In [14]:
def scrap_matches_by_formats(links_list):
    df = pd.DataFrame()
    match_links = []
    for link in links_list:
        html_text = requests.get(link)
        soup = BeautifulSoup(html_text.content,"html.parser")
        table = soup.find("table",class_ = "engineTable")
        columns = []
        for th in table.find('thead').find_all('th'):
            columns.append(th.text.strip())
        data = []
        for tr in table.find('tbody').find_all('tr'):
            row = []
            for td in tr.find_all('td'):
                row.append(td.text.strip())
                if td.find('a'):
                    link = td.find('a')['href']
                    if 'match' in link:
                        match_links.append("https://stats.espncricinfo.com" + link)
            if row[2] == 'no result': 
                if match_links:
                    match_links.pop() 
                continue 
            data.append(row)
        columns = ['Team 1','Team 2','Winner','Margin','Ground','Match Date','Match id']   
        for row in data:
            match = row[0] + " vs " + row[1]
            row.insert(0, match)
            df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
    return df, match_links

In [15]:
matches_df,match_links = scrap_matches_by_formats(year_links)

/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)


/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)


/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)
/tmp/ipykernel_2963/2462605207.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(row, index=['Match'] + columns), ignore_index=True)


In [18]:
matches_df.head()

,Match,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match id
0,West Indies vs Ireland,West Indies,Ireland,West Indies,24 runs,Kingston,"Jan 8, 2022",ODI # 4339
1,West Indies vs Ireland,West Indies,Ireland,Ireland,5 wickets,Kingston,"Jan 13, 2022",ODI # 4340
2,Sri Lanka vs Zimbabwe,Sri Lanka,Zimbabwe,Sri Lanka,5 wickets,Pallekele,"Jan 16, 2022",ODI # 4341
3,West Indies vs Ireland,West Indies,Ireland,Ireland,2 wickets,Kingston,"Jan 16, 2022",ODI # 4342
4,Sri Lanka vs Zimbabwe,Sri Lanka,Zimbabwe,Zimbabwe,22 runs,Pallekele,"Jan 18, 2022",ODI # 4343


In [22]:
matches_df.to_csv("matches_for_world_cup_model.csv",index=False)

In [19]:
def scrape_batting_and_bowling_data(match_url):
    batting_df = pd.DataFrame()
    bowling_df = pd.DataFrame()
    
    html_content = requests.get(match_url)
    soup = BeautifulSoup(html_content.content,"html.parser")
    try:
        batting_and_bowling = soup.find_all("div",class_ = "ds-p-0")[1:3]
        match_number_div = soup.find_all("div",class_ = "ds-p-0")[3]
        match_number = match_number_div.find_all("span",class_ = "ds-text-tight-s ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block")[-1].text.split(" ")[-1]
    #     print(match_number)
        innings_batting_data = []
        innings_bowling_data = []
        inningsTeam = soup.find_all("span",class_ = "ds-text-title-xs ds-font-bold ds-capitalize")
        team1 = inningsTeam[0].text
        team2 = inningsTeam[1].text
        match_name = team1 + " Vs"  + team2
        print(f"Processing match: {match_name} with match id: {match_number}")
        for div in batting_and_bowling:
            innings_team_name = inningsTeam.pop(0).text
            table = div.find_all("table")
            innings_batting = table[0]
            batting_columns = []
            for th in innings_batting.find("thead").find_all('th'):
                batting_columns.append(th.text.strip())
            batting_data = []
            for tr in innings_batting.find("tbody").find_all('tr'):
                batting_row = []
                for td in tr.find_all("td"):
                    batting_row.append(td.text.strip())
                if any(batting_row):
                    batting_data.append(batting_row) 
            batting_columns = ['Batsman Name','Dismissal','Runs','Balls','M','4s','6s','Strike Rate']
            innings_batting_df = pd.DataFrame(batting_data, columns=batting_columns)
            innings_batting_df = innings_batting_df[~innings_batting_df['Batsman Name'].str.contains('Extras|TOTAL|Did not bat|Fall of wickets')]
            innings_batting_df.insert(0,"Match Name",match_name)
            innings_batting_df.insert(1, "teamInnings", innings_team_name)
            innings_batting_df.insert(2, "Batting Position", range(1, 1+len(innings_batting_df)))
            innings_batting_df.insert(10,"Match id",match_number)
            batting_df = batting_df.append(innings_batting_df, ignore_index=True)

            #bowling scorecard
            innings_bowling = table[1]
            bowling_columns = []
            for th in innings_bowling.find("thead").find_all('th'):
                bowling_columns.append(th.text.strip())
            bowling_data = []
            for tr in innings_bowling.find("tbody").find_all('tr'):
                bowling_row = []
                for td in tr.find_all("td"):
                    bowling_row.append(td.text.strip())
                if len(bowling_row)!=1:
                    bowling_data.append(bowling_row) 
            bowling_columns = ['Bowler Name','Over','Maiden','Runs','Wickets','Economy','dots','fours','sixes','wides','no balls']
            innings_bowling_df = pd.DataFrame(bowling_data, columns=bowling_columns)
            if innings_team_name == team1:
                innings_team_name = team2
            else:
                innings_team_name = team1
            innings_bowling_df.insert(0, "match", match_name)
            innings_bowling_df.insert(1, "teamInnings", innings_team_name)
            innings_bowling_df.insert(13,"Match Id",match_number)
            bowling_df = bowling_df.append(innings_bowling_df, ignore_index=True)
    except:
        pass
        
    return batting_df,bowling_df

In [21]:
combined_batting_df = pd.DataFrame()
combined_bowling_df = pd.DataFrame()

# Loop through the match URLs
for match_url in match_links:
    batting_df, bowling_df = scrape_batting_and_bowling_data(match_url)
    combined_batting_df = combined_batting_df.append(batting_df)
    combined_bowling_df = combined_bowling_df.append(bowling_df)

# Write the combined batting and bowling dataframes to CSV files
combined_batting_df.to_csv('batting_data_for_world_cup.csv', index=False)
combined_bowling_df.to_csv('bowling_data_for_world_cup.csv', index=False)

Processing match: West Indies VsIreland with match id: 4339
Processing match: West Indies VsIreland with match id: 4340
Processing match: Zimbabwe VsSri Lanka with match id: 4341
Processing match: West Indies VsIreland with match id: 4342
Processing match: Zimbabwe VsSri Lanka with match id: 4343
Processing match: South Africa VsIndia with match id: 4344
Processing match: Afghanistan VsNetherlands with match id: 4345
Processing match: India VsSouth Africa with match id: 4346
Processing match: Sri Lanka VsZimbabwe with match id: 4347
Processing match: Afghanistan VsNetherlands with match id: 4348
Processing match: South Africa VsIndia with match id: 4349
Processing match: Afghanistan VsNetherlands with match id: 4350
Processing match: Oman VsUnited Arab Emirates with match id: 4351
Processing match: Oman VsUnited Arab Emirates with match id: 4352
Processing match: West Indies VsIndia with match id: 4353
Processing match: Oman VsUnited Arab Emirates with match id: 4354
Processing match: 

Processing match: South Africa VsIndia with match id: 4471
Processing match: South Africa VsIndia with match id: 4472
Processing match: United Arab Emirates VsNepal with match id: 4473
Processing match: United Arab Emirates VsNepal with match id: 4474
Processing match: England VsAustralia with match id: 4475
Processing match: United Arab Emirates VsNepal with match id: 4476
Processing match: Australia VsEngland with match id: 4477
Processing match: Papua New Guinea VsNamibia with match id: 4478
Processing match: United States of America VsNamibia with match id: 4479
Processing match: Australia VsEngland with match id: 4480
Processing match: Papua New Guinea VsUnited States of America with match id: 4481
Processing match: Papua New Guinea VsNamibia with match id: 4482
Processing match: India VsNew Zealand with match id: 4483
Processing match: United States of America VsPapua New Guinea with match id: 4484
Processing match: Afghanistan VsSri Lanka with match id: 4485
Processing match: Un

In [4]:
matches_df = pd.read_csv("matches_for_world_cup_model.csv")

In [5]:
matches_df.head()

,Match,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match id
0,West Indies vs Ireland,West Indies,Ireland,West Indies,24 runs,Kingston,"Jan 8, 2022",ODI # 4339
1,West Indies vs Ireland,West Indies,Ireland,Ireland,5 wickets,Kingston,"Jan 13, 2022",ODI # 4340
2,Sri Lanka vs Zimbabwe,Sri Lanka,Zimbabwe,Sri Lanka,5 wickets,Pallekele,"Jan 16, 2022",ODI # 4341
3,West Indies vs Ireland,West Indies,Ireland,Ireland,2 wickets,Kingston,"Jan 16, 2022",ODI # 4342
4,Sri Lanka vs Zimbabwe,Sri Lanka,Zimbabwe,Zimbabwe,22 runs,Pallekele,"Jan 18, 2022",ODI # 4343


In [6]:
teams = ["India","Australia","Afghanistan","England","South Africa","New Zealand","Zimbabwe","West Indies","Ireland","Sri Lanka","Pakistan","Bangladesh"]

In [7]:
len(teams)

12

In [8]:
matches_df = matches_df[(matches_df["Team 1"].isin(teams)) & (matches_df["Team 2"].isin(teams))]

In [10]:
matches_df.shape

(98, 8)

In [15]:
matches_df.to_csv("matches_for_world_cup_model.csv",index=False)
matches_df["Match id"] = matches_df["Match id"].str.split("#").str[1]

In [16]:
matches_df.head()

,Match,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match id
0,West Indies vs Ireland,West Indies,Ireland,West Indies,24 runs,Kingston,"Jan 8, 2022",4339
1,West Indies vs Ireland,West Indies,Ireland,Ireland,5 wickets,Kingston,"Jan 13, 2022",4340
2,Sri Lanka vs Zimbabwe,Sri Lanka,Zimbabwe,Sri Lanka,5 wickets,Pallekele,"Jan 16, 2022",4341
3,West Indies vs Ireland,West Indies,Ireland,Ireland,2 wickets,Kingston,"Jan 16, 2022",4342
4,Sri Lanka vs Zimbabwe,Sri Lanka,Zimbabwe,Zimbabwe,22 runs,Pallekele,"Jan 18, 2022",4343


In [14]:
batting_df = pd.read_csv("batting_data_for_world_cup.csv")
batting_df.head()

,Match Name,teamInnings,Batting Position,Batsman Name,Dismissal,Runs,Balls,M,4s,6s,Match id,Strike Rate
0,West Indies VsIreland,West Indies,1,Shai Hope,c †Tucker b Campher,29,44,87,3,0,4339,65.90
1,West Indies VsIreland,West Indies,2,Justin Greaves,c Delany b Adair,7,24,25,1,0,4339,29.16
2,West Indies VsIreland,West Indies,3,Nicholas Pooran †,c Porterfield b McBrine,13,34,58,2,0,4339,38.23
3,West Indies VsIreland,West Indies,4,Shamarh Brooks,lbw b Adair,93,89,132,9,3,4339,104.49
4,West Indies VsIreland,West Indies,5,Roston Chase,c Tector b McBrine,1,4,8,0,0,4339,25.00


In [18]:
batting_df["Match id"] = batting_df["Match id"].astype(int)
matches_df["Match id"] = matches_df["Match id"].astype(int)
merged_batting_df = pd.merge(batting_df,matches_df,on = "Match id")


In [20]:
merged_batting_df = merged_batting_df[["Match Name","teamInnings","Batting Position","Batsman Name","Dismissal","Runs","Balls","4s","6s","Match id","Strike Rate"]]

In [21]:
merged_batting_df.to_csv("batting_data_for_world_cup.csv",index=False)

In [22]:
bowling_df = pd.read_csv("bowling_data_for_world_cup.csv")
bowling_df.head()

,match,teamInnings,Bowler Name,Over,Maiden,Runs,Wickets,Economy,dots,fours,sixes,wides,no balls,Match Id
0,West Indies VsIreland,Ireland,Josh Little,10.0,1,46,1,4.60,34,4,1,0,0,4339
1,West Indies VsIreland,Ireland,Mark Adair,8.5,1,38,3,4.30,36,4,1,3,0,4339
2,West Indies VsIreland,Ireland,Craig Young,9.0,0,56,3,6.22,29,8,0,5,0,4339
3,West Indies VsIreland,Ireland,Curtis Campher,10.0,0,47,1,4.70,39,2,2,4,0,4339
4,West Indies VsIreland,Ireland,Andy McBrine,8.0,0,52,2,6.50,26,3,4,0,0,4339


In [26]:
bowling_df["Match Id"] = bowling_df["Match Id"].astype(int)
bowling_df = bowling_df.rename(columns={"Match Id":"Match id"})

In [27]:
merged_bowling_df = pd.merge(bowling_df,matches_df,on = "Match id")

In [29]:
merged_bowling_df = merged_bowling_df[["match","teamInnings","Bowler Name","Over","Maiden","Runs","Wickets","Economy","dots","fours","sixes","wides","no balls","Match id"]]

In [30]:
merged_bowling_df.to_csv("bowling_data_for_world_cup.csv",index=False)